In [1]:
import sys
from gurobipy import *
import numpy as np
import math
import time
from numpy import linalg as LA

In [2]:
# f = open("1-4-multi-100-1.txt", "r")
# print(f.read())


In [3]:
# print(f.read())

In [4]:
# f = open("1-4-multi-100-1.txt", "r")
# print(f.read())

# Set initial values
epsilon = 0.20

In [5]:
filename = '1-7-1-1000-3.txt'
f = open("1-7-1-1000-3.txt", "r")
a = []
for line in f:
    a.append(line)

# # The data is of the list type.  The Python list type is actually
# # a dynamic array. The lines contain also the \n; hence the .rstrip()
# for n, line in enumerate(data, 1):
#     print '{:2}.'.format(n), line.rstrip()

# print '-----------------'

In [6]:
# a

In [7]:
c = a[0]
c = c.replace("[", "")
c = c.replace("]\n", "")
c = c.split(",")
c = np.array(c)
c = c.astype(np.float64)

In [8]:
b = a[-1]
b = b.replace("[", "")
b = b.replace("]\n", "")
b = b.split(",")
b = np.array(b)
b = b.astype(np.float64)

In [9]:
xi = [None]*(len(a)-2)

In [10]:
for i in range(1,len(a)-1):
    dd = a[i]
    dd = dd.replace("[", "")
    dd = dd.replace("],\n", "")
    dd = dd.replace("]]\n", "")
    dd = dd.split(",")
    dd = np.array(dd)
    dd = dd.astype(np.float64)
    xi[i-1] = dd
    
    

In [11]:
np.size(c)

50

In [12]:
np.size(b)

1000

In [13]:
np.size(xi)

50000

In [14]:
np.size(xi[0])

50

In [15]:
random_size = np.size(b)
x_random_size = np.size(c)

In [16]:
k=math.floor(random_size*epsilon)

In [17]:
c = -c

In [18]:
# CVaR Method

def Model4():
    #Model
    print ("Begin to solve Model CVaR")

    m = Model()
    #m.setParam('TimeLimit', 60*60)
    # Create variables
    m.update()
    # s1 = m.addVars(random_size,name="s1")
    x = m.addVars(x_random_size,lb=0,ub=1,name="x")
    s = m.addVars(random_size,lb=-GRB.INFINITY,name="s")
#     s = m.addVars(random_size,lb=-GRB.INFINITY,name="s")
  #  z = m.addVars(random_size,name="z")
    beta = m.addVar(lb=-GRB.INFINITY,ub=0,name="beta")

    m.update()
        
    
    # Add objective    
    m.setObjective(sum(c[i]*x[i] for i in range(x_random_size)), GRB.MINIMIZE)
    m.update()
    # Add constraints
    m.addConstrs(  s[i] >= beta for i in range(random_size))
    m.update()

   
    m.addConstrs( sum(xi[i][j]*x[j] for j in range(x_random_size)) <= b[i] + s[i]  for i in range(random_size))
    m.update()
    m.addConstr( s.sum()/random_size - (1-epsilon)*beta <=0)
    m.update()
    # Solve the problem
    m.update()
        
    m.optimize()
    

#     Store solutions
#     ppp = m.getAttr('x', beta)
    CVAR_s = m.getAttr('x', s)
    CVAR_x = m.getAttr('x', x)
    
#     for i in range(random_size):
#      #   g_x_si[i] = g_x + si[i] 
#         c_x = 0
#         for j in range(x_random_size):
#             c_x += a[i,j]*kkk[j]
#         c_x_xi[i] = c_x  
    
#     for i in range(random_size):
#         if c_x_xi[i]-4 <= 0:
#             z[i] = 1
#         else:
#             z[i] = 0
# #     return pp,ii,kk
#     print('Obj: %g' % m.objVal)
    aaaa=m.objVal
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    return m.objVal,CVAR_s,CVAR_x
    
    


In [19]:
start=time.time()
CVaR=Model4()
CVAR=CVaR[0]
CVAR_s=CVaR[1]
CVAR_x=CVaR[2]
modeltime_cvar= time.time() - start


Begin to solve Model CVaR
Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-06
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2001 rows, 1051 columns and 49053 nonzeros
Model fingerprint: 0xf88777a1
Coefficient statistics:
  Matrix range     [1e-03, 4e+02]
  Objective range  [2e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+02, 8e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 2001 rows, 1049 columns, 49053 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 49
 AA' NZ     : 4.905e+04
 Factor NZ  : 9.895e+04 (roughly 2 MB of memory)
 Factor Ops : 4.832e+06 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal       

In [20]:
# quantitle bounds

def Model_q():

    print ("Begin to solve model quantitle relaxtion")

    m = Model()
    
    m.setParam('TimeLimit', 60*60)
    # Create variables
    m.update()
    # s1 = m.addVars(random_size,name="s1")
    x = m.addVars(x_random_size,lb=0,ub=1, name="x")

    m.update()
    
    # Set functions
    
    #obj = 0
    g_x_xi = [None] * random_size
    f_x = 0
    g_x = 0
    

    m.update()
    # Set functions
    for i in range(x_random_size):
      
        f_x += c[i]*x[i]
        m.update()
    
#     for i in range(random_size):
#         g_x_si[i] = g_x +       SSS   
    for i in range(random_size):
     #   g_x_si[i] = g_x + si[i] 
        g_x = 0
        for j in range(x_random_size):
            g_x += xi[i][j]*x[j]
            m.update()
        g_x_xi[i] = g_x  
    
    # Add objective    
    m.setObjective(f_x, GRB.MINIMIZE)
    m.update()
    # Add constraints
    
    con1=m.addConstr(g_x_xi[0] <= b[0] )
    # Solve the problem
    m.update()
    m.params.OutputFlag=0    
    m.optimize()
    kkk = m.getAttr('x', x)
    result = [0]*random_size
    obj = 0
    for i in range(x_random_size):
        obj += c[i]*kkk[i]
    result[0]=obj
    
    
    for i in range(1,random_size):   
        m.remove(con1)
        con1=m.addConstr(g_x_xi[i]  <= b[i] )
        m.update()
        m.params.OutputFlag=0    
        m.optimize()
        kkk = m.getAttr('x', x)
        obj = 0
        for j in range(x_random_size):
            obj += c[j]*kkk[j]
        result[i]=obj
    return result
        



In [21]:
start=time.time()
quantile_result=Model_q()
sorted_array=np.sort(quantile_result,-1)
v_q = sorted_array[random_size-k-1]
print(v_q)
modeltime_quantile = time.time() - start

Begin to solve model quantitle relaxtion
Set parameter TimeLimit to value 3600
-17182.225652656874


In [22]:
v_q

-17182.225652656874

In [23]:
CVAR

-16719.163292912002

In [24]:
violation = math.floor(random_size*epsilon)

In [25]:
CVAR

-16719.163292912002

In [26]:
# CVaR Method

def Model_alsox_sharp():
    #Model
    print ("Begin to solve model ALSOX Sharp")
    delta_1 = 1e-1
    v_lower_upper = v_q
    v_upper_bound = CVAR
    current_bound = (v_lower_upper+v_upper_bound)/2.0
    
    delta_t = v_upper_bound - v_lower_upper
    
    m = Model()
    #m.setParam('TimeLimit', 60*60)
    # Create variables
    m.update()
    # s1 = m.addVars(random_size,name="s1")
    x = m.addVars(x_random_size,lb=0,ub=1,name="x")
    s = m.addVars(random_size,lb=-GRB.INFINITY,name="s")
  #  z = m.addVars(random_size,name="z")
    beta = m.addVar(lb=-GRB.INFINITY,ub=0,name="beta")
 
    m.update()
    

    # Add objective    
    m.setObjective(s.sum()/random_size - (1-epsilon)*beta, GRB.MINIMIZE)
    m.update()
    # Add constraints
    m.addConstrs(  s[i] >= beta for i in range(random_size))
    m.update()
#     m.addConstr(slack_t*slack_t >= sum(x[i]*x[i] for i in range(x_random_size)))
#     m.addConstrs(slack_t >= x[i] for i in range(x_random_size))
    m.update()

   
    m.addConstrs( sum(xi[i][j]*x[j] for j in range(x_random_size)) <= b[i] + s[i]  for i in range(random_size))
    m.update()
    con1=m.addConstr(sum(c[i]*x[i] for i in range(x_random_size)) <= current_bound)
    m.update()
    # Solve the problem
    m.update()
    
    m.params.OutputFlag=0
        
    m.optimize()
        
#     Store solutions
#     ppp = m.getAttr('x', beta)
    current_s = m.getAttr('x', s)
    current_x = m.getAttr('x', x)
    
    
    support=0
    for j in current_s:
        if current_s[j]>0:
            support +=1
    if support > violation:
        v_lower_upper = current_bound
#         t1=con1.SARHSUp
    else:
        v_upper_bound = current_bound
    current_bound = (v_lower_upper + v_upper_bound)/2.0
    delta_t = v_upper_bound - v_lower_upper
    con1.RHS = current_bound
    for i in range(random_size):
        s[i].start = current_s[i]
    for i in range(x_random_size):
        x[i].start = current_x[i]  
    m.optimize()
    current_s = m.getAttr('x', s)
    current_x = m.getAttr('x', x)

    
    
    
#     aaaa=m.objVal
    
#     for v in m.getVars():
#         print('%s %g' % (v.varName, v.x))
    return v_upper_bound,current_x
    
    


In [27]:
start=time.time()    
alsox_sharp = Model_alsox_sharp()
modeltime_alsox_sharp = time.time() - start   

Begin to solve model ALSOX Sharp


In [28]:
v_q

-17182.225652656874

In [29]:
(CVAR-v_q)/abs(v_q)

0.026950080222771903

In [30]:
(alsox_sharp[0]-v_q)/abs(v_q)

0.013475040111385951

In [31]:
alsox_sharp[0]

-16950.69447278444

In [32]:
CVAR

-16719.163292912002

In [33]:
#  lower bound improvment
start=time.time()   
delta_1 = 1e-1
current_bound = v_q
# v_upper_bound = CVAR
# current_bound = -20000

delta_t = 10


m = Model()
#m.setParam('TimeLimit', 60*60)
# Create variables
m.update()
# s1 = m.addVars(random_size,name="s1")
x = m.addVars(x_random_size,lb=0,ub=1,name="x")

y = m.addVar(lb=-GRB.INFINITY,name="y")

s = m.addVars(random_size,lb=0,name="s")

beta = m.addVar(lb=-GRB.INFINITY,ub=0,name="beta")


slack_t = m.addVar(lb=0,name="slack_t")

mu = m.addVars(x_random_size,random_size,lb=0,ub=1, name="mu")

w = m.addVars(x_random_size,random_size,lb=0,ub=1, name="w")

# z = m.addVars(random_size,vtype=GRB.BINARY,name="z")

z = m.addVars(random_size,lb=0,ub=1,name="z")

m.update()

m.setObjective(y, GRB.MINIMIZE)
# Add objective 
# m.addConstr( slack_t >= sum(x[i] for i in range(x_random_size)))
# m.addConstrs( slack_t >= x[i] for i in range(x_random_size))
m.update()


m.addConstrs( sum(xi[i][j]*mu[j,i] for j in range(x_random_size)) <= b[i]*z[i] for i in range(random_size))
m.update()

# Solve the problem
# m.update()

m.addConstrs( x[i] == mu[i,j] + w[i,j] for i in range(x_random_size) for j in range(random_size) )
m.update()
# m.addConstrs( mu[j,i]<=1-z[j]  for j in range(x_random_size) for i in range(random_size) )
m.addConstrs( w[j,i] <= 1 - z[i]  for j in range(x_random_size) for i in range(random_size) )
m.update()
m.addConstrs( mu[j,i] <= z[i] for j in range(x_random_size) for i in range(random_size))
m.update()
m.addConstr(z.sum() >= random_size - math.ceil(random_size*epsilon))
m.update()

constr1 = m.addConstrs( y >= sum(c[i] * mu[i,j] for i in range(x_random_size)) + current_bound * (1-z[j])  for j in  range(random_size))
m.update()
constr2 = m.addConstrs( y >= sum(c[i] * w[i,j] for i in range(x_random_size)) + current_bound * (z[j])  for j in  range(random_size) )
m.update()

m.params.OutputFlag=0
m.params.BarHomogeneous=1 
m.optimize()

# current_s = m.getAttr('x', s)
current_x = m.getAttr('x', x)

dual_bound = m.objVal
print('bound is ', dual_bound)

ite = 0 

while delta_t >= delta_1:
    
    m.remove(m.getConstrs()[-(2*random_size)-1:-1])
#     m.remove(m.getConstrs()[-2])
    delta_t = abs(dual_bound - current_bound)
    
    current_bound = dual_bound
    
#     if m.status == 2:
#         current_bound = dual_bound
#     else:
#         v_lower_upper = current_bound

#     current_bound = (v_lower_upper + v_upper_bound)/2.0
    
#     print('current bound is', current_bound)
    constr1 = m.addConstrs( y >= sum(c[i] * mu[i,j] for i in range(x_random_size)) + current_bound * (1-z[j])  for j in  range(random_size))
    m.update()
    constr2 = m.addConstrs( y >= sum(c[i] * w[i,j] for i in range(x_random_size)) + current_bound * (z[j])  for j in  range(random_size) )
    m.update()
#     constr1.RHS = current_bound
#     constr2.RHS = current_bound
#     for i in range(random_size):
#         s[i].start = current_s[i]
    for i in range(x_random_size):
        x[i].start = current_x[i]  
    m.params.OutputFlag=0
    print('current bound is', current_bound)
    m.optimize()
    dual_bound = m.objVal
    
    ite = ite + 1 
    
current_bound = dual_bound
print('with iterations', ite)
#     Store solutions
#     ppp = m.getAttr('x', beta)
# first_dual_bound_s = m.getAttr('x', s)
# first_dual_bound_x = m.getAttr('x', x)

# dual_bound = m.objVal

modeltime_second_dual_bound = time.time() - start   


bound is  -17156.85629901042
current bound is -17156.85629901042
current bound is -17156.85629901042
with iterations 2


In [34]:
lower_bound_second_dual = current_bound

In [35]:
value_approximation_bound = alsox_sharp[0]
# value_approximation_bound = min(CVAR,alsox_sharp_result[0])
# value_approximation_bound = CVAR

In [36]:
sorted_array_indice = [i for i, value in enumerate(quantile_result) if value > value_approximation_bound]


In [37]:
cut_z_indice = sorted_array_indice

In [38]:
stengthen_difference = [element for element in range(random_size) if element not in cut_z_indice]


In [39]:
len(cut_z_indice)

91

In [40]:
subset_indice = np.argsort(quantile_result)[random_size-2*k:random_size]
# subset_indice = np.argsort(scenario_value_alsox_sharp)[random_size-2*k:random_size]
stengthen_difference_update = [element for element in subset_indice if element not in cut_z_indice]
stengthen_difference_update=stengthen_difference_update[::-1]

In [41]:
# start=time.time()
# cut_third_one = []

# # value_approximation_bound = value_alsox
# print ("Begin to solve second dual bound")

# m = Model()
# m.setParam('Seed', 2)
# m.setParam('TimeLimit', 60*60)
# # Create variables
# m.update()
# # s1 = m.addVars(random_size,name="s1")
# x = m.addVars(x_random_size,lb=0,ub=1, name="x")
# mu = m.addVars(x_random_size,random_size,lb=0,ub=1, name="mu")
# w = m.addVars(x_random_size,random_size,lb=0,ub=1, name="w")
# z = m.addVars(random_size,lb=0,ub=1,name="z")
# y = m.addVar(lb=-GRB.INFINITY,name="y")


# m.update()



# # Add objective    
# m.setObjective(y, GRB.MINIMIZE)
# m.update()
# #     m.addConstr(z[3]==1)
# #     m.addConstr(z[4]==1)
# #     m.addConstr(z[5]==1)
# # constr1 = m.addConstrs( sum(c[i] * mu[i,j] for i in range(x_random_size)) <= value_approximation_bound * (z[j])  for j in stengthen_difference)
# # m.update()
# # constr2 = m.addConstrs( sum(c[i] * w[i,j] for i in range(x_random_size)) <= value_approximation_bound * (1-z[j])  for j in stengthen_difference )
# # m.update()

# # Add constraints
# m.addConstrs( x[i] == mu[i,j]+w[i,j] for i in range(x_random_size) for j in stengthen_difference )
# m.update()
# # m.addConstrs( mu[j,i]<=1-z[j]  for j in range(x_random_size) for i in range(random_size) )
# m.addConstrs( w[j,i]<=1-z[i]  for j in range(x_random_size) for i in stengthen_difference )
# m.update()
# m.addConstrs( mu[j,i] <= z[i] for j in range(x_random_size) for i in stengthen_difference)
# m.update()
# # m.addConstrs( sum( xi[i][j]*mu[j,i] for j in range(x_random_size)) <= b[i]*(z[i]) for i in range(random_size)) 
# # m.update()
# # m.addConstrs(sum(xi[i][j]*x[j]  for j in range(x_random_size)) >= b[i] for i in cut_z_indice)

# m.addConstrs(sum( xi[i][j]*mu[j,i] for j in range(x_random_size)) <= b[i] for i in stengthen_difference) 
# m.update()
# # constr3 = m.addConstrs( sum( xi[i][j]*w[j,i] for j in range(x_random_size)) <= b[i]*z[i] for i in range(random_size))
# m.update()
# #     m.addConstrs(0 <= x[i] <= 1 for i in range(x_random_size))
# m.addConstr(z.sum() >= math.ceil(random_size*(1-epsilon)))

# m.addConstr(y >= sum(c[i] * x[i] for i in range(x_random_size)))

# m.update()


# m.addConstrs(z[i] == 0 for i in cut_z_indice)

# m.addConstrs( y >= sum(c[i] * mu[i,j] for i in range(x_random_size)) + value_approximation_bound * (1-z[j])  for j in stengthen_difference)
# m.update()
# m.addConstrs( y >= sum(c[i] * w[i,j] for i in range(x_random_size)) + value_approximation_bound * (z[j])  for j in stengthen_difference)
# m.update()

# m.update()

# m.params.OutputFlag=0 

# m.optimize()

# x_solution_store_1 = m.getAttr('x', z)
# mu_solution_store_1 = m.getAttr('x', mu)
# w_solution_store_1 = m.getAttr('x', w)
# x_solution_store_1 = m.getAttr('x', x)
# z_solution_store_1 = m.getAttr('x', z)

# m.remove(m.getConstrs()[-(2*len(stengthen_difference))-1:-1])

# iteration = 0

# for i in range(0,math.ceil(k*0.1)):       


#         print(stengthen_difference_update[i])
        
#         m.addConstr(z[stengthen_difference_update[i]]==1)
#         m.update()

#         stengthen_difference_new = [element for element in range(random_size) if element not in cut_z_indice and element!=stengthen_difference_update[i]]

#         m.addConstrs( y >= sum(c[i] * mu[i,j] for i in range(x_random_size)) + value_approximation_bound * (1-z[j])  for j in stengthen_difference_new)
#         m.update()
#         m.addConstrs( y >= sum(c[i] * w[i,j] for i in range(x_random_size)) + value_approximation_bound * (z[j])  for j in stengthen_difference_new)
#         m.update()


#         x.start = x_solution_store_1 
#         mu.start = mu_solution_store_1
#         w.start = w_solution_store_1
#         z.start = z_solution_store_1

#         m.params.OutputFlag=0 

#         m.optimize()

#         print('Updated obj is', m.ObjVal)
        
#         if m.objVal > value_approximation_bound:
            
#     #     print(m.status)
# #         if m.status == 4:
# #         if m.status == 12 or m.status == GRB.INFEASIBLE:
#             print("Cut Found")
#             cut_third_one.append([i])
        
#         iteration += 1
        
#         m.remove(m.getConstrs()[-(2*len(stengthen_difference_new))-2:-1])


# modeltime_fixing_1 = time.time() - start





In [42]:
modeltime_fixing_1 = 0
cut_third_one = []

In [43]:
for i in range(len(cut_third_one)):
    cut_z_indice.append(stengthen_difference_update[cut_third_one[i][0]])
    


In [44]:
subset_indice = np.argsort(quantile_result)[random_size-2*k:random_size]
stengthen_difference_update = [element for element in subset_indice if element not in cut_z_indice]
stengthen_difference_update=stengthen_difference_update[::-1]

In [45]:
# start=time.time()
# cut_third = []
# # value_approximation_bound = value_alsox
# print ("Begin to solve second dual bound")

# m = Model()
# m.setParam('Seed', 2)
# m.setParam('TimeLimit', 60*60)
# # Create variables
# m.update()
# # s1 = m.addVars(random_size,name="s1")
# x = m.addVars(x_random_size,lb=0,ub=1, name="x")
# mu = m.addVars(x_random_size,random_size,lb=0,ub=1, name="mu")
# w = m.addVars(x_random_size,random_size,lb=0,ub=1, name="w")
# z = m.addVars(random_size,lb=0,ub=1,name="z")
# y = m.addVar(lb=-GRB.INFINITY,name="y")


# m.update()



# # Add objective    
# m.setObjective(y, GRB.MINIMIZE)
# m.update()
# #     m.addConstr(z[3]==1)
# #     m.addConstr(z[4]==1)
# #     m.addConstr(z[5]==1)

# # Add constraints
# m.addConstrs( x[i] == mu[i,j]+w[i,j] for i in range(x_random_size) for j in stengthen_difference )
# m.update()
# # m.addConstrs( mu[j,i]<=1-z[j]  for j in range(x_random_size) for i in range(random_size) )
# m.addConstrs( w[j,i]<=1-z[i]  for j in range(x_random_size) for i in stengthen_difference )
# m.update()
# m.addConstrs( mu[j,i] <= z[i] for j in range(x_random_size) for i in stengthen_difference)
# m.update()
# # m.addConstrs( sum( xi[i][j]*mu[j,i] for j in range(x_random_size)) <= b[i]*(z[i]) for i in range(random_size)) 
# # m.update()
# # m.addConstrs(sum(xi[i][j]*x[j]  for j in range(x_random_size)) >= b[i] for i in cut_z_indice)

# m.addConstrs(sum( xi[i][j]*mu[j,i] for j in range(x_random_size)) <= b[i] for i in stengthen_difference) 
# m.update()
# # constr3 = m.addConstrs( sum( xi[i][j]*w[j,i] for j in range(x_random_size)) <= b[i]*z[i] for i in range(random_size))
# m.update()
# #     m.addConstrs(0 <= x[i] <= 1 for i in range(x_random_size))
# m.addConstr(z.sum() >= math.ceil(random_size*(1-epsilon)))

# # m.addConstrs(slack_t[0] - slack_t_mu[j] - slack_t_w[j] == 0 for j in range(random_size))
# # 

# m.addConstr(y >= sum(c[i] * x[i] for i in range(x_random_size)))

# m.update()


# m.addConstrs(z[i] == 0 for i in cut_z_indice)

# m.addConstrs( y >= sum(c[i] * mu[i,j] for i in range(x_random_size)) + value_approximation_bound * (1-z[j])  for j in stengthen_difference)
# m.update()
# m.addConstrs( y >= sum(c[i] * w[i,j] for i in range(x_random_size)) + value_approximation_bound * (z[j])  for j in stengthen_difference)
# m.update()

# m.update()

# m.params.OutputFlag=0 

# m.optimize()

# z_solution_store_1 = m.getAttr('x', z)
# mu_solution_store_1 = m.getAttr('x', mu)
# w_solution_store_1 = m.getAttr('x', w)
# x_solution_store_1 = m.getAttr('x', x)

# print('current obj is', m.ObjVal)


# m.remove(m.getConstrs()[-(2*len(stengthen_difference))-1:-1])

# iteration = 0

# for i in range(0,math.ceil(k*0.2)):       

#     for j in range(1,2):
        
#         print(stengthen_difference_update[i],stengthen_difference_update[i+j])
        
#         m.addConstr(z[stengthen_difference_update[i]]+z[stengthen_difference_update[i+j]]==2)
#         m.update()

#         stengthen_difference_new = [element for element in range(random_size) if element not in cut_z_indice and element!=stengthen_difference_update[i] and element!=stengthen_difference_update[i+j]]

#         m.addConstrs( y >= sum(c[i] * mu[i,j] for i in range(x_random_size)) + value_approximation_bound * (1-z[j])  for j in stengthen_difference_new)
#         m.update()
#         m.addConstrs( y >= sum(c[i] * w[i,j] for i in range(x_random_size)) + value_approximation_bound * (z[j])  for j in stengthen_difference_new)
#         m.update()


#         x.start = x_solution_store_1 
#         mu.start = mu_solution_store_1
#         w.start = w_solution_store_1
#         z.start = z_solution_store_1

#         m.params.OutputFlag=0 

#         m.optimize()

#         print('Updated obj is', m.ObjVal)
        
#         if m.objVal > value_approximation_bound:
            
#     #     print(m.status)
# #         if m.status == 4:
# #         if m.status == 12 or m.status == GRB.INFEASIBLE:
#             print("Cut Found")
#             cut_third.append([i,i+j])
        
#         iteration += 1
        
#         m.remove(m.getConstrs()[-(2*len(stengthen_difference_new))-2:-1])


# modeltime_fixing_2 = time.time() - start


In [46]:
cut_third = []

In [47]:
len(cut_third)

0

In [48]:
modeltime_fixing_2 = 0

In [49]:
modeltime_fixing = modeltime_fixing_1 + modeltime_fixing_2

In [50]:
len(cut_z_indice)

91

In [51]:
number_of_cuts = len(cut_z_indice) + len(cut_third)

In [52]:
alsox_solution=alsox_sharp[1]

In [53]:
start=time.time()
Big_M_updated = [0]*random_size
for j in stengthen_difference:
    eta_value=[]
    print('current j is', j)
    for i in stengthen_difference:
        if sum(xi[i]) <= b[i]:
            eta_value.append(max(sum(xi[j]) - b[j],0))
#             print("Special:",i)
        else:  
            division_sort = xi[i][np.argsort(np.divide(xi[j],xi[i]))][::-1]

            division_index = np.argsort(np.divide(xi[j],xi[i]))[x_random_size-np.where(division_sort.cumsum() > b[i])[0][0]:x_random_size]

            eta_value.append(max(sum(xi[j][division_index]) - b[j],0))
#             print(i)
    Big_M_updated[j]=eta_value[np.argsort(eta_value)[len(stengthen_difference)-math.floor(random_size*epsilon)+len(cut_z_indice)+1]]
modeltime_big_m_stengthen= time.time() - start



current j is 0
current j is 1
current j is 2
current j is 3
current j is 4
current j is 5
current j is 6
current j is 7
current j is 8
current j is 9
current j is 10
current j is 11
current j is 12
current j is 13
current j is 14
current j is 15
current j is 16
current j is 17
current j is 18


/var/folders/x7/9lcnbn_11xg08cv6gk733_ch0000gn/T/ipykernel_58375/1675386552.py:11: RuntimeWarning: invalid value encountered in divide
  division_sort = xi[i][np.argsort(np.divide(xi[j],xi[i]))][::-1]
/var/folders/x7/9lcnbn_11xg08cv6gk733_ch0000gn/T/ipykernel_58375/1675386552.py:13: RuntimeWarning: invalid value encountered in divide
  division_index = np.argsort(np.divide(xi[j],xi[i]))[x_random_size-np.where(division_sort.cumsum() > b[i])[0][0]:x_random_size]
/var/folders/x7/9lcnbn_11xg08cv6gk733_ch0000gn/T/ipykernel_58375/1675386552.py:11: RuntimeWarning: divide by zero encountered in divide
  division_sort = xi[i][np.argsort(np.divide(xi[j],xi[i]))][::-1]
/var/folders/x7/9lcnbn_11xg08cv6gk733_ch0000gn/T/ipykernel_58375/1675386552.py:13: RuntimeWarning: divide by zero encountered in divide
  division_index = np.argsort(np.divide(xi[j],xi[i]))[x_random_size-np.where(division_sort.cumsum() > b[i])[0][0]:x_random_size]


current j is 20
current j is 21
current j is 22
current j is 23
current j is 25
current j is 26
current j is 27
current j is 28
current j is 29
current j is 31
current j is 32
current j is 33
current j is 34
current j is 35
current j is 36
current j is 37
current j is 38
current j is 39
current j is 40
current j is 41
current j is 42
current j is 43
current j is 44
current j is 45
current j is 46
current j is 47
current j is 48
current j is 49
current j is 51
current j is 52
current j is 53
current j is 54
current j is 55
current j is 57
current j is 58
current j is 59
current j is 60
current j is 61
current j is 63
current j is 65
current j is 66
current j is 68
current j is 69
current j is 70
current j is 71
current j is 73
current j is 74
current j is 75
current j is 76
current j is 77
current j is 78
current j is 79
current j is 81
current j is 82
current j is 83
current j is 84
current j is 85
current j is 86
current j is 87
current j is 88
current j is 89
current j is 90
current 

current j is 563
current j is 564
current j is 565
current j is 566
current j is 567
current j is 568
current j is 569
current j is 570
current j is 571
current j is 572
current j is 573
current j is 574
current j is 575
current j is 576
current j is 577
current j is 578
current j is 579
current j is 580
current j is 581
current j is 582
current j is 583
current j is 584
current j is 585
current j is 586
current j is 587
current j is 588
current j is 589
current j is 590
current j is 591
current j is 592
current j is 593
current j is 594
current j is 595
current j is 596
current j is 597
current j is 599
current j is 600
current j is 601
current j is 602
current j is 603
current j is 604
current j is 605
current j is 606
current j is 607
current j is 608
current j is 609
current j is 610
current j is 611
current j is 612
current j is 613
current j is 615
current j is 617
current j is 619
current j is 620
current j is 621
current j is 622
current j is 624
current j is 625
current j is 6

In [54]:
# Big-M Method
#Big_M = [None] * random_size
# Big_M = []
# for i in range(random_size):
   
#     Big_M.append(math.ceil(sum(x for x in a[i,:] if x > 0) -bb[i])) 
# Big_M = [30]*random_size

def Model3_var():
    print ("Begin to solve model 3")
    
    m = Model()
    m.setParam('Seed', 2)
    m.setParam('TimeLimit', 4*60*60)
    # Create variables
    m.update()
    # s1 = m.addVars(random_size,name="s1")
    x = m.addVars(x_random_size,lb=0,ub=1,name="x")
    z = m.addVars(stengthen_difference,vtype=GRB.BINARY,name="z")
#     y = m.addVars(stengthen_difference,lb=-GRB.INFINITY,name="y")
#     s = m.addVars(stengthen_difference,lb=-GRB.INFINITY,name="s")
#     gamma = m.addVars(1,lb=0,name="gamma")
   
    m.update()

    # Set functions
    
    #obj = 0
#     g_x_xi = [None] * len(z)
#     f_x = 0
#     g_x = 0
    
    # Set functions
#     for i in range(x_random_size):
      
#         f_x += c[i]*x[i]
#         m.update()
    
#     for i in range(random_size):
#         g_x_si[i] = g_x +       SSS   
#     for i in range(random_size):
#      #   g_x_si[i] = g_x + si[i] 
#         g_x = 0
#         for j in range(x_random_size):
#             g_x += xi[i][j]*x[j]
#             m.update()
#         g_x_xi[i] = g_x  
    
    # Add objective    
    m.setObjective(sum(c[i]*x[i]  for i in range(x_random_size)), GRB.MINIMIZE)
    m.update()
    
    m.addConstr(sum(c[i]*x[i]  for i in range(x_random_size)) <= value_approximation_bound)
    
    m.update()
    
    m.addConstr(sum(c[i]*x[i]  for i in range(x_random_size)) >= lower_bound_second_dual)
    
    m.update()
    
    m.update()
    # Add constraints
   
    m.addConstrs(sum(xi[i][j]*x[j]  for j in range(x_random_size)) >= b[i] for i in cut_z_indice)
    
    m.update()
    
    m.addConstrs( sum(xi[i][j]*x[j]  for j in range(x_random_size)) <= b[i]   + Big_M_updated[i]*z[i] for i in stengthen_difference)
    m.update()


    m.addConstr(z.sum() <= math.floor(random_size*epsilon)-len(cut_z_indice))
    m.update()
#     m.addConstrs(z[stengthen_difference_update[cut_third[i][0]]]+z[stengthen_difference_update[cut_third[i][1]]]>= 1 for i in range(len(cut_third)))

    # Solve the problem
    m.update()
    
#     for i in range(x_random_size):

#         x[i].start = alsox_solution[i]
    
#     x.start = alsox_sharp[1]
    
    
#     m.params.BarHomogeneous=1     
    m.optimize()
    
    # Store solutions
    #pp = m.getAttr('x', z)
    z_VaR = m.getAttr('x', z)
    x_VaR = m.getAttr('x', x)
    
    #return pp,ii,kk
    
    return m.objVal,z_VaR,x_VaR
    

In [ ]:
start=time.time()
Model3_var_value = Model3_var()
z_VaR = Model3_var_value[1]
x_VaR = Model3_var_value[2]
# value_z = Model3_cut()[0]
# kk=Model3_cut()[1]
# obj_cut = 0
# for i in range(x_random_size):
#     obj_cut += c[i]*kk[i]
modeltime_Model3_var = time.time() - start
# print(f'The value of optimal value t is exactly {obj_cut_stengthen_updated:.5f}.')
print(f'The value of running time is approximately {modeltime_Model3_var:.3f} s.')
# print(f'The value of total running time is approximately {modeltime_cut_stengthen_updated+modeltime_big_m_stengthen+modeltime_fixing:.3f} s.')


Begin to solve model 3
Set parameter Seed to value 2
Set parameter TimeLimit to value 14400
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1003 rows, 959 columns and 46968 nonzeros
Model fingerprint: 0xa6345366
Variable types: 50 continuous, 909 integer (909 binary)
Coefficient statistics:
  Matrix range     [6e-01, 4e+03]
  Objective range  [2e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 2e+04]
Presolve removed 1 rows and 3 columns
Presolve time: 0.03s
Presolved: 1002 rows, 956 columns, 46922 nonzeros
Variable types: 49 continuous, 907 integer (907 binary)

Root relaxation: objective -1.715686e+04, 283 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -17156.856    0  107          - -17156.856

 2309103 544498     cutoff  120      -17013.395 -17055.866  0.25%  10.7  320s
 2347634 550779 -17036.202  103   95 -17013.395 -17055.498  0.25%  10.6  325s
 2384396 558641     cutoff  112      -17013.395 -17055.177  0.25%  10.6  330s
 2422888 565693 -17028.671  100  115 -17013.395 -17054.809  0.24%  10.6  335s
 2460349 573336 -17018.886  118   51 -17013.395 -17054.417  0.24%  10.5  340s
 2500145 580862 -17021.149   74   96 -17013.395 -17054.074  0.24%  10.5  345s
 2537128 586595 infeasible  116      -17013.395 -17053.722  0.24%  10.5  350s
 2573439 593339 -17038.961   96   94 -17013.395 -17053.389  0.24%  10.5  355s
 2612151 600919     cutoff  111      -17013.395 -17053.122  0.23%  10.4  360s
 2650441 606541 infeasible  118      -17013.395 -17052.778  0.23%  10.4  365s
 2691165 614797 infeasible  121      -17013.395 -17052.433  0.23%  10.4  370s
 2730609 621414 infeasible  118      -17013.395 -17052.162  0.23%  10.3  375s
 2776091 630243 -17049.636   86  143 -17013.395 -17051.835  0.23

In [ ]:
modeltime_preprocessing = modeltime_quantile + modeltime_cvar + modeltime_second_dual_bound + modeltime_alsox_sharp

In [ ]:
print(f'The value of modeltime preprocessing is approximately {modeltime_preprocessing:.3f} s.')


In [ ]:
print(f'The value of modeltime modeltime_big_m_stengthen is approximately {modeltime_big_m_stengthen:.3f} s.')


In [ ]:
print(f'The value of modeltime fixing is approximately {modeltime_fixing:.3f} s.')


In [ ]:
print(f'The number of cuts is approximately {number_of_cuts:.3f} s.')


In [ ]:
start=time.time()
Big_M_strength_only = []
for j in range(random_size):
    eta_value=[]
    for i in range(random_size):
        if sum(xi[i]) <= b[i]:
            eta_value.append(max(sum(xi[j]) - b[j],0))
#             print("Special:",i)
        else:  
            division_sort = xi[i][np.argsort(np.divide(xi[j],xi[i]))][::-1]

            division_index = np.argsort(np.divide(xi[j],xi[i]))[x_random_size-np.where(division_sort.cumsum() > b[i])[0][0]:x_random_size]

            eta_value.append(max(sum(xi[j][division_index]) - b[j],0))
#             print(i)
    Big_M_strength_only.append(eta_value[np.argsort(eta_value)[random_size-math.floor(random_size*epsilon)+1]])
modeltime_big_m_stengthen_only = time.time() - start


In [ ]:
# Big-M Method
#Big_M = [None] * random_size
# Big_M = []
# for i in range(random_size):
   
#     Big_M.append(math.ceil(sum(x for x in a[i,:] if x > 0) -bb[i])) 
# Big_M = [30]*random_size

def Model3_strength_only():
    #Model
    print ("Begin to solve model 3")

    m = Model()
    m.setParam('Seed', 2)
    m.setParam('TimeLimit', 4*60*60)
    # Create variables
    m.update()
    # s1 = m.addVars(random_size,name="s1")
    x = m.addVars(x_random_size,lb=0,ub=1,name="x")
    z = m.addVars(random_size,vtype=GRB.BINARY,name="z")
#     y = m.addVars(stengthen_difference,lb=-GRB.INFINITY,name="y")
#     s = m.addVars(stengthen_difference,lb=-GRB.INFINITY,name="s")
#     gamma = m.addVars(1,lb=0,name="gamma")
#     lambda_1 = m.addVars(1,lb=0,name="lambda_1")
   
    m.update()

    # Set functions
    
    #obj = 0
#     g_x_xi = [None] * len(z)
#     f_x = 0
#     g_x = 0
    
    # Set functions
#     for i in range(x_random_size):
      
#         f_x += c[i]*x[i]
#         m.update()
    
#     for i in range(random_size):
#         g_x_si[i] = g_x +       SSS   
#     for i in range(random_size):
#      #   g_x_si[i] = g_x + si[i] 
#         g_x = 0
#         for j in range(x_random_size):
#             g_x += xi[i][j]*x[j]
#             m.update()
#         g_x_xi[i] = g_x  
    
    # Add objective    
    m.setObjective(sum(c[i]*x[i]  for i in range(x_random_size)), GRB.MINIMIZE)
    m.update()
    
#     m.addConstr(sum(c[i]*x[i]  for i in range(x_random_size)) >= v_q)
    

   
#     m.addConstrs(sum(xi[i][j]*x[j]  for j in range(x_random_size)) >= b[i] for i in cut_z_indice)
#     m.update()
    
    m.addConstrs(sum(xi[i][j]*x[j]  for j in range(x_random_size)) <= b[i]   + Big_M_strength_only[i]*z[i] for i in range(random_size))
    m.update()


    m.addConstr(z.sum() <= math.floor(random_size*epsilon))
    m.update()
#     m.addConstrs(z[stengthen_difference_update[cut_third[i][0]]]+z[stengthen_difference_update[cut_third[i][1]]]>= 1 for i in range(len(cut_third)))

    # Solve the problem
    m.update()
#     m.params.BarHomogeneous=1     
    m.optimize()
    
    # Store solutions
    #pp = m.getAttr('x', z)
    z_strength_only = m.getAttr('x', z)
    x_strength_only = m.getAttr('x', x)
    
    #return pp,ii,kk
    
    return m.objVal,z_strength_only,x_strength_only

In [ ]:
start=time.time()
Model3_strength_only_value = Model3_strength_only()
# z_VaR = Model3_var_value[1]
# x_VaR = Model3_var_value[2]
# value_z = Model3_cut()[0]
# kk=Model3_cut()[1]
# obj_cut = 0
# for i in range(x_random_size):
#     obj_cut += c[i]*kk[i]
modeltime_Model3_strength_only = time.time() - start
# print(f'The value of optimal value t is exactly {obj_cut_stengthen_updated:.5f}.')
print(f'The value of running time is approximately {modeltime_Model3_strength_only:.3f} s.')
# print(f'The value of total running time is approximately {modeltime_cut_stengthen_updated+modeltime_big_m_stengthen+modeltime_fixing:.3f} s.')


In [ ]:
print(f'The value of Big-M stengthening only is approximately {modeltime_big_m_stengthen_only:.3f} s.')

In [ ]:
Big_M = []
for i in range(random_size):
   
    Big_M.append(math.ceil(sum(x for x in xi[i][:] if x > 0) -b[i])) 
# Big_M = [30]*random_size


In [ ]:
# Big-M Method
#Big_M = [None] * random_size
# Big_M = []
# for i in range(random_size):
   
#     Big_M.append(math.ceil(sum(x for x in a[i,:] if x > 0) -bb[i])) 
# Big_M = [30]*random_size

def Model3():
    #Model
    print ("Begin to solve model 3")

    m = Model()
    m.setParam('Seed', 2)
    m.setParam('TimeLimit', 4*60*60)
    # Create variables
    m.update()
    # s1 = m.addVars(random_size,name="s1")
    x = m.addVars(x_random_size,lb=0,ub=1,name="x")
    z = m.addVars(random_size,vtype=GRB.BINARY,name="z")
   
#     y = m.addVars(stengthen_difference,lb=-GRB.INFINITY,name="y")
#     s = m.addVars(stengthen_difference,lb=-GRB.INFINITY,name="s")
#     gamma = m.addVars(1,lb=0,name="gamma")
#     lambda_1 = m.addVars(1,lb=0,name="lambda_1")
   
    m.update()

    # Set functions
    
    #obj = 0
#     g_x_xi = [None] * len(z)
#     f_x = 0
#     g_x = 0
    
    # Set functions
#     for i in range(x_random_size):
      
#         f_x += c[i]*x[i]
#         m.update()
    
#     for i in range(random_size):
#         g_x_si[i] = g_x +       SSS   
#     for i in range(random_size):
#      #   g_x_si[i] = g_x + si[i] 
#         g_x = 0
#         for j in range(x_random_size):
#             g_x += xi[i][j]*x[j]
#             m.update()
#         g_x_xi[i] = g_x  
    
    # Add objective    
    m.setObjective(sum(c[i]*x[i]  for i in range(x_random_size)), GRB.MINIMIZE)
    m.update()
    # Add constraints
    
    
    m.addConstrs( sum(xi[i][j]*x[j]  for j in range(x_random_size)) <= b[i]   + Big_M[i]*z[i] for i in range(random_size))
    m.update()


    m.addConstr(z.sum() <= math.floor(random_size*epsilon))
    m.update()

    # Solve the problem
    m.update()
#     m.params.BarHomogeneous=1     
    m.optimize()
    
    # Store solutions
    #pp = m.getAttr('x', z)
    z_org= m.getAttr('x', z)
    x_org = m.getAttr('x', x)
    
    #return pp,ii,kk
    
    return m.objVal,z_org,x_org


In [ ]:
start=time.time()
Model3_big=Model3()
obj_big_m = Model3_big[0]
value_z = Model3_big[1]
kk=Model3_big[2]
# obj = 0
# for i in range(x_random_size):
#     obj += c[i]*kk[i]
modeltime_big_m = time.time() - start
print(f'The value of optimal value t is exactly {obj_big_m:.5f}.')
print(f'The value of running time is approximately {modeltime_big_m:.3f} s.')
